# Zoning tagger

In [33]:
import geopandas as gpd
from enum import Enum
import sys
from pathlib import Path
import plotly.graph_objects as go

# Добавляем путь к папке my_package в sys.path
project_path = Path("C:/repos/master-plan-requirements")
sys.path.append(str(project_path))

## PZZ data

ОПРЕДЛЕНИЕ ЗОНИРОВАНИЯ КВАРТАЛОВ

In [2]:
# загружаем подготовленный слой
pzz_gdf = gpd.read_file('./data/pzz_gdf.geojson')
pzz_gdf.head()

,Зона,geometry
0,ТПД2_2/Т1Ж1,"POLYGON ((3350357.896 8404319.308, 3350391.347..."
1,ТПД2_2/Т1Ж1,"POLYGON ((3350357.383 8404317.109, 3350357.632..."
2,ТД1-1_1,"POLYGON ((3373712.579 8385251.612, 3373743.594..."
3,ТД2_1,"POLYGON ((3373939.959 8384632.388, 3373939.901..."
4,ТД2_1,"POLYGON ((3373827.838 8384502.448, 3373823.108..."


In [3]:
# Импортируем теггер зонирования
from my_package.methods.zoning_tagger_1 import ZoneTagger
from my_package.models.tag import Tag

In [4]:
tagger = ZoneTagger(pzz_gdf)
zoning_tag = tagger.run()

2025-04-09 21:25:02.916 | INFO     | my_package.methods.zoning_tagger_1:run:59 - Assigning zoning tags from column 'Зона'
2025-04-09 21:25:02.923 | INFO     | my_package.methods.zoning_tagger_1:run:68 - Removed 126 geometries without zoning tag
2025-04-09 21:25:02.924 | SUCCESS  | my_package.methods.zoning_tagger_1:run:74 - Zoning tags assigned


РАСЧЕТ ПЕРЕХОДА ЗОН

In [5]:
# Импортируем матрицу и теги
from my_package.models.transition_matrix import TRANSITION_MATRIX
from my_package.models.tag import Tag

In [6]:
TRANSITION_MATRIX

,Tag.RESIDENTIAL,Tag.PUBLIC_AND_BUSINESS,Tag.INDUSTRIAL,Tag.ENGINEERING_AND_TRANSPORTATION,Tag.RECREATIONAL,Tag.AGRICULTURAL,Tag.SPECIAL_PURPOSE
Tag.RESIDENTIAL,1.0000,0.5730,0.2759,0.4000,0.4898,0.3770,0.2105
Tag.PUBLIC_AND_BUSINESS,0.7612,1.0000,0.5517,0.5556,0.8163,0.4262,0.3158
Tag.INDUSTRIAL,0.2388,0.3596,1.0000,0.7333,0.2041,0.4754,0.4211
Tag.ENGINEERING_AND_TRANSPORTATION,0.2687,0.2809,0.5690,1.0000,0.1633,0.2459,0.4737
Tag.RECREATIONAL,0.3582,0.4494,0.1724,0.1778,1.0000,0.2459,0.1579
Tag.AGRICULTURAL,0.3433,0.2921,0.5000,0.3333,0.3061,1.0000,0.3158
Tag.SPECIAL_PURPOSE,0.0597,0.0674,0.1379,0.2000,0.0612,0.0984,1.0000


In [7]:
from my_package.methods.get_transition import TRANSITION_RESULT
zoning_tag = TRANSITION_RESULT(zoning_tag)

2025-04-09 21:25:07.309 | INFO     | my_package.methods.get_transition:TRANSITION_RESULT:51 - Fixing geometry...
2025-04-09 21:25:07.542 | INFO     | my_package.methods.get_transition:TRANSITION_RESULT:58 - Creating transition columns...
2025-04-09 21:25:07.792 | INFO     | my_package.methods.get_transition:TRANSITION_RESULT:65 - Buffering geometry to find neighbors...
2025-04-09 21:25:10.046 | INFO     | my_package.methods.get_transition:TRANSITION_RESULT:69 - Computing transition values using neighbors...
2025-04-09 21:25:22.889 | INFO     | my_package.methods.get_transition:TRANSITION_RESULT:75 - Updating transition columns with neighborhood-adjusted values...
2025-04-09 21:25:22.899 | INFO     | my_package.methods.get_transition:TRANSITION_RESULT:83 - Normalizing and rounding transition values...
2025-04-09 21:25:23.597 | SUCCESS  | my_package.methods.get_transition:TRANSITION_RESULT:88 - Transition matrix computation complete.


РАСЧЕТ ТЕХНИКО-ЭКОНОМИЧЕСКИХ ПОКАЗАТЕЛЕЙ

In [10]:
from my_package.methods.get_TEP import TEP_RESULT
zoning_tag = TEP_RESULT(zoning_tag)

2025-04-09 21:27:00.739 | INFO     | my_package.methods.get_TEP:TEP_RESULT:10 - Fixing geometry...
2025-04-09 21:27:00.973 | INFO     | my_package.methods.get_TEP:TEP_RESULT:17 - Calculating area for each block...
2025-04-09 21:27:00.978 | INFO     | my_package.methods.get_TEP:TEP_RESULT:21 - Calculating total city area...
2025-04-09 21:27:00.978 | INFO     | my_package.methods.get_TEP:TEP_RESULT:25 - Calculating total area for each zoning tag...
2025-04-09 21:27:00.986 | INFO     | my_package.methods.get_TEP:TEP_RESULT:30 - Computing block area as percentage of city...
2025-04-09 21:27:00.986 | INFO     | my_package.methods.get_TEP:TEP_RESULT:34 - Computing block area as percentage of its zone...
2025-04-09 21:27:01.024 | SUCCESS  | my_package.methods.get_TEP:TEP_RESULT:44 - TEP indicators successfully calculated.


In [14]:
# Общая площадь города
total_city_area = zoning_tag['area'].sum()

# Площадь каждой зоны (группировка по строковому представлению zoning_tag)
zone_areas = zoning_tag.groupby(zoning_tag['zoning_tag'].astype(str))['area'].sum().to_dict()

# Вывод общей площади города
print(f"Общая площадь города: {total_city_area / 1_000_000:.2f} км²")

# Вывод площадей зон
zone_areas_str = "\n".join([f"{zone}: {area / 1_000_000:.2f} км²" for zone, area in zone_areas.items()])
print(f"Площадь зон:\n{zone_areas_str}")

Общая площадь города: 5034.54 км²
Площадь зон:
Tag.AGRICULTURAL: 335.81 км²
Tag.ENGINEERING_AND_TRANSPORTATION: 817.66 км²
Tag.INDUSTRIAL: 154.37 км²
Tag.PUBLIC_AND_BUSINESS: 464.78 км²
Tag.RECREATIONAL: 1810.93 км²
Tag.RESIDENTIAL: 1334.78 км²
Tag.SPECIAL_PURPOSE: 116.22 км²


In [34]:
# Преобразуем значения из м² в км² и округляем
zone_areas_km2 = {zone: round(area / 1_000_000, 2) for zone, area in zone_areas.items()}

In [37]:
from my_package.models.tag import Tag

# Преобразуем строковые ключи 'Tag.RESIDENTIAL' → Tag.RESIDENTIAL → 'Жилая зона'
zone_labels = [
    Tag[key.split('.')[-1]].value if key.split('.')[-1] in Tag.__members__ else key
    for key in zone_areas.keys()
]
zone_values = list(zone_areas.values())

# Значения в км² и проценты
zone_values_km2 = [round(value / 1_000_000, 2) for value in zone_areas.values()]
zone_percentages = [(value / total_city_area) * 100 for value in zone_areas.values()]

In [38]:
# Всплывающие подсказки
custom_hover = [
    f"{label}: {area:.2f} км² ({percent:.1f}%)"
    for label, area, percent in zip(zone_labels, zone_values_km2, zone_percentages)
]

# Создаём кольцевую диаграмму
fig = go.Figure(data=[go.Pie(
    labels=zone_labels,
    values=zone_values_km2,
    textinfo='none',        # без текста в секторе
    hovertext=custom_hover,
    hoverinfo='text',
    hole=0.4                # белый круг в центре
)])

fig.update_layout(title_text="Распределение площади по зонам")
fig.show()

In [39]:
zoning_tag

,geometry,zoning_tag,Tag.RESIDENTIAL_transition,Tag.PUBLIC_AND_BUSINESS_transition,Tag.INDUSTRIAL_transition,Tag.ENGINEERING_AND_TRANSPORTATION_transition,Tag.RECREATIONAL_transition,Tag.AGRICULTURAL_transition,Tag.SPECIAL_PURPOSE_transition,area,area_city,area_zone
0,"POLYGON ((3350357.896 8404319.308, 3350391.347...",Tag.ENGINEERING_AND_TRANSPORTATION,0.0448,0.0468,0.0948,0.6665,0.0272,0.0410,0.0789,1.132897e+05,0.002250,0.013855
1,"POLYGON ((3350357.383 8404317.109, 3350357.632...",Tag.ENGINEERING_AND_TRANSPORTATION,0.0192,0.0201,0.0406,0.8571,0.0117,0.0176,0.0338,2.564143e+06,0.050931,0.313596
2,"POLYGON ((3373712.579 8385251.612, 3373743.594...",Tag.PUBLIC_AND_BUSINESS,0.0944,0.3722,0.0685,0.0689,0.3039,0.0529,0.0392,3.562697e+04,0.000708,0.007665
3,"POLYGON ((3373939.959 8384632.388, 3373939.901...",Tag.PUBLIC_AND_BUSINESS,0.1452,0.1907,0.1052,0.1060,0.3114,0.0813,0.0602,2.093329e+03,0.000042,0.000450
4,"POLYGON ((3373827.838 8384502.448, 3373823.108...",Tag.PUBLIC_AND_BUSINESS,0.0824,0.5409,0.0597,0.0601,0.1766,0.0461,0.0342,1.197619e+04,0.000238,0.002577
...,...,...,...,...,...,...,...,...,...,...,...,...
7157,"POLYGON ((3336747.174 8419985.493, 3336773.6 8...",Tag.RECREATIONAL,0.0972,0.1220,0.0234,0.0241,0.6785,0.0334,0.0214,4.721771e+04,0.000938,0.002607
7158,"POLYGON ((3336214.332 8419562.903, 3336318.864...",Tag.RECREATIONAL,0.0546,0.0685,0.0263,0.0271,0.7620,0.0375,0.0241,2.403590e+03,0.000048,0.000133
7159,"POLYGON ((3338995.718 8402909.162, 3339000.462...",Tag.RESIDENTIAL,0.2893,0.2072,0.0200,0.0868,0.3542,0.0273,0.0152,6.172420e+06,0.122601,0.462430
7160,"POLYGON ((3359348.393 8401121.685, 3359358.978...",Tag.RECREATIONAL,0.0782,0.0490,0.0188,0.0194,0.7637,0.0537,0.0172,2.729093e+07,0.542073,1.507009


In [17]:
# Сохраняем в GeoJSON
zoning_tag.to_file(
    r"C:\repos\master-plan-requirements\examples\data\zoning_tag.geojson", driver="GeoJSON", encoding="utf-8")

СЦЕНАРИЙ 1 - ИЗМЕНЕНИЕ ЗОНИРОВАНИЯ

In [35]:
selected_indices = [13600, 785, 782, 14298, 13599, 13743]
# Извлечение полигонов с указанными индексами и столбцом 'tags'
selected_polygons = zoning_gdf.loc[selected_indices, ['geometry', 'tags']]
# Шаг 2: Преобразуем столбец 'tags' в строковый тип, если это необходимо
selected_polygons['tags'] = selected_polygons['tags'].astype(str)  # Преобразуем в строку
# Убираем префикс 'Tag.' из значений в столбце 'tags'
selected_polygons['tags'] = selected_polygons['tags'].str.replace('Tag.', '', regex=False)

In [38]:
selected_polygons
selected_polygons.to_file('selected_polygons.geojson', driver='GeoJSON')

,geometry,tags
13600,"POLYGON ((324286.424 6643162.192, 324308.983 6...",[<RESIDENTIAL: 'Жилая зона'>]
785,"POLYGON ((324460.162 6642203.399, 324475.843 6...",[<RECREATIONAL: 'Рекреационная'>]
782,"POLYGON ((323125.302 6642836.055, 323109.807 6...",[<RECREATIONAL: 'Рекреационная'>]
14298,"POLYGON ((324225.134 6643018.167, 324221.631 6...",[<RESIDENTIAL: 'Жилая зона'>]
13599,"POLYGON ((324410.599 6643457.317, 324400.664 6...",[<RESIDENTIAL: 'Жилая зона'>]
13743,"POLYGON ((324219.587 6643722.397, 324234.364 6...",[<RECREATIONAL: 'Рекреационная'>]


In [98]:
selected_polygons.explore()

In [21]:
selected_indices_2 = [6773, 4538,6071,5504,484,486,4550]
# Извлечение полигонов с указанными индексами и столбцом 'tags'
selected_polygons_2 = zoning_gdf.loc[selected_indices_2, ['geometry', 'tags']]

# Шаг 2: Преобразуем столбец 'tags' в строковый тип, если это необходимо
selected_polygons_2['tags'] = selected_polygons_2['tags'].astype(str)  # Преобразуем в строку
# Убираем префикс 'Tag.' из значений в столбце 'tags'
selected_polygons_2['tags'] = selected_polygons_2['tags'].str.replace('Tag.', '', regex=False)
selected_polygons_2
selected_polygons_2.to_file('selected_polygons_2.geojson', driver='GeoJSON')
selected_polygons_2.explore()

In [23]:
selected_indices_3 = [78, 77,2831,2832,2483,210,190,8121,8794,491,494,107]
# Извлечение полигонов с указанными индексами и столбцом 'tags'
selected_polygons_3 = zoning_gdf.loc[selected_indices_3, ['geometry', 'tags']]

# Шаг 2: Преобразуем столбец 'tags' в строковый тип, если это необходимо
selected_polygons_3['tags'] = selected_polygons_3['tags'].astype(str)  # Преобразуем в строку
# Убираем префикс 'Tag.' из значений в столбце 'tags'
selected_polygons_3['tags'] = selected_polygons_3['tags'].str.replace('Tag.', '', regex=False)
selected_polygons_3
selected_polygons_3.to_file('selected_polygons_3.geojson', driver='GeoJSON')
selected_polygons_3.explore()

In [112]:
import geopandas as gpd

poly_prob = gpd.read_file('selected_polygons_prob.geojson')

In [113]:
poly_prob

,tags,probability,geometry
0,[RESIDENTIAL],1.000,"POLYGON ((29.85953 59.88851, 29.85988 59.88901..."
1,[RECREATIONAL],0.349,"POLYGON ((29.86344 59.87999, 29.86373 59.87990..."
2,[RECREATIONAL],0.349,"POLYGON ((29.83909 59.88509, 29.83881 59.88515..."
3,[RESIDENTIAL],1.000,"POLYGON ((29.85856 59.88719, 29.85850 59.88712..."
4,[RESIDENTIAL],1.000,"POLYGON ((29.86149 59.89121, 29.86134 59.89099..."
5,[RECREATIONAL],0.349,"POLYGON ((29.85786 59.89350, 29.85804 59.89437..."


In [103]:
import pandas as pd
import geopandas as gpd

selected_polygons
compatibility_matrix = pd.read_csv('matrix.csv', index_col=0)

# Шаг 3: Найдем тег полигона с ID 13600
target_polygon_id = 13600
target_polygon = selected_polygons[selected_polygons.index == target_polygon_id]  # выбираем полигон с ID 13600
if not target_polygon.empty:
    target_tag = target_polygon['tags'].iloc[0][0]  # предполагаем, что это список, и берем первый элемент
else:
    raise ValueError(f"Полигон с ID {target_polygon_id} не найден!")

# Шаг 4: Добавим столбец 'probability' для всех полигонов в GDF
probabilities = []

for idx, row in selected_polygons.iterrows():
    zone = row['tags'][0]  # извлекаем зону для текущего полигона
    # Извлекаем значение вероятности из матрицы сочетания для текущего полигона и целевого полигона
    if target_tag in compatibility_matrix.columns and zone in compatibility_matrix.index:
        probability = compatibility_matrix.loc[zone, target_tag]
    else:
        probability = 0  # Если сочетание не найдено в матрице, присваиваем 0
    probabilities.append(probability)
# Шаг 5: Добавим новый столбец 'probability' в GeoDataFrame
selected_polygons['probability'] = probabilities
# Шаг 6: Сохраним результат в новый файл GeoJSON
selected_polygons.to_file('output_with_probabilities.geojson', driver='GeoJSON')

# Выводим первые строки для проверки
print(selected_polygons[['tags', 'probability']].head())